In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

train = pd.read_csv('train_process.csv')
test = pd.read_csv('test_process.csv')

C:\Users\User\Anaconda3\envs\kaggle\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [13]:
X_train = train.drop("Survived", axis=1)
y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()
X_train.shape, y_train.shape, X_test.shape

((891, 6), (891,), (418, 6))

## 높은 정확도를 보였던 3개의 model을 ensemble하여 평가
* SVM, Random Forest, Logistic Regression

In [9]:
#library import

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [27]:
log_clf=LogisticRegression(solver='liblinear')
random_clf = RandomForestClassifier(n_estimators=10)
svc_clf = SVC(gamma='auto', probability=True)

In [28]:
voting_clf=VotingClassifier(
        estimators=[('lr',log_clf), ('rf', random_clf), ('svc', svc_clf)],
        voting='soft')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=Non...
                                        

## Models 결과 비교

In [29]:
# 각각의 classifier, ensemble모델의 정확도 평가
from sklearn.model_selection import cross_val_score


for clf in (log_clf, random_clf, svc_clf, voting_clf):
    clf.fit(X_train, y_train)
    acc_clf=cross_val_score(clf, X_train, y_train, cv=5)
    print('cross-val-score.mean \n{:.3f}'.format(acc_clf.mean()))

cross-val-score.mean 
0.794
cross-val-score.mean 
0.796
cross-val-score.mean 
0.822
cross-val-score.mean 
0.815


In [30]:
prediction=voting_clf.predict(X_test)

In [32]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })
submission.to_csv('titanic_ensemble.csv', index=False)